base de dados:
- dados dos posts wordpress (pre-migração)
- dados dos posts wix (pós-migração)

Como organizar os dados de forma que facilite a migração e tratamento entre as duas plataformas?

- usar duckdb para compilar os dados relacionado a cada post
- 

In [3]:
import duckdb

# Connect to an in-memory database
# con = duckdb.connect(database=':memory:', read_only=False)

# Or connect to a file
con = duckdb.connect(database='data/migration.duckdb', read_only=False)

# Create a sample table for demonstration
con.execute("CREATE TABLE items(id INTEGER, name VARCHAR);");
con.execute("INSERT INTO items VALUES (1, 'Laptop'), (2, 'Mouse'), (3, 'Keyboard');");

# Query to show columns from a specific table
table_name = 'items'

# Method 1: Using DESCRIBE
print(f"Columns in '{table_name}' using DESCRIBE:")
describe_result = con.execute(f"DESCRIBE {table_name}").fetchall()
for col in describe_result:
    print(f"- {col[0]} ({col[1]})")

print('\n' + '-'*30 + '\n')

# Method 2: Using PRAGMA table_info
print(f"Columns in '{table_name}' using PRAGMA table_info:")
pragma_result = con.execute(f"PRAGMA table_info('{table_name}')").fetchall()
# The result format is: (column_id, column_name, column_type, nullable, default_value, is_primary_key)
for col in pragma_result:
    print(f"- {col[1]} ({col[2]})")


# Close the connection
con.close()


Columns in 'items' using DESCRIBE:
- id (INTEGER)
- name (VARCHAR)

------------------------------

Columns in 'items' using PRAGMA table_info:
- id (INTEGER)
- name (VARCHAR)


An error occurred: Catalog Error: Table with name my_table does not exist!
Did you mean "pg_tables"?

LINE 1: SELECT * FROM "my_table" LIMIT 5
                      ^
